
# データ構造化事例　＃２
## 粉末・薄膜用のXRDデータのデータ構造化 



**対応機種** ：　 リガク　SmartLab  
**rawデータ**：　ras (テキスト形式)  
**スクリプトの内容**：  
主に粉体・薄膜用のXRD測定に用いられるリガク（SmartLab）のデータについて，データ構造化（数値csvファイル化）ならびに回折図を出力します．

![image](https://user-images.githubusercontent.com/38028745/134605660-c22bdca9-51f9-463b-8e59-83765d38a2d6.png)

<hr>

## はじめに　Google ColabでGitHubのコードを動かす準備　（Google Colabでなければ不要）
Google ColabでGitHubのコードを動かすための方法です．  
**Google Colabからではない場合には本作業はスキップ**してください．  

次のボタンを押してGoogle Colabにアクセスします．  
もしグレーアウトで開かない場合には**右クリック**で「**新しいタブで開く**」を選んで進んでください．

<a href="https://colab.research.google.com/github/ARIM-Japan/Training_Program/blob/main/XRD_Rigaku_Smartlab_Training.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## サンプルデータを読み込みましょう

In [1]:
!git clone https://github.com/ARIM-Japan/Training_Program.git
%cd Training_Program

C:\Users\Shigeyuki Matsunami\Dropbox\4_データ設計\1_データ構造化\Github\Training_Program\Training_Program


Cloning into 'Training_Program'...


これで準備が整いました．

<hr>

## 1. ライブラリーを読み込みましょう
最初にコードを走らせるのに必要なライブラリーを読み込みます．ライブラリとしては以下を使っています．
 * 標準ライブラリー: ``glob``, ``os``, ``csv``
 * 数値処理用: ``pandas``
 * 可視化用: ``matplotlib``  

In [1]:
# ファイル操作用
import sys
import glob
import os
import io
import re
import csv

# 数値処理用
import pandas as pd
from chardet import detect

また，出力結果を「output」フォルダーに保存するため，そのフォルダーの作成をします．

In [2]:
output_folder = 'output'
os.makedirs(output_folder,exist_ok = True)

## 2. ファイルの拡張子を調べます
データ構造化の最初のステップとして，データ構造化するファイルを特定する必要があります．その特定は**拡張子**で判定します．  
下記の``read_files``関数は入力ファイルが置いてある「data」フォルダーにある特定の拡張子のファイルをすべてリスト化します．ここでは単一のファイルではなく複数のファイルが扱えることを想定しています．

もしエラーがでたら「data」フォルダーを作成し，その中に.txtファイルを配置してみてください．

In [3]:
# 実行処理#rawデータの読み込み
def read_files(extension):
    """
    概要: 特定の拡張子のついたファイルをすべて読み込む
    @param extension: 読み込む拡張子
    @return ソートされたファイル名群
    """

    #入力データのフォルダーを"data"とする
    data_folder = 'data/*'
    path = data_folder + extension
    
    # 連続撮影されたファイル名の確認
    files = glob.glob(path)
    
    input_files = sorted(files)
    print (input_files)
    
    return input_files

SEMで撮影された撮影情報ファイルである.txtを含むファイルを指定してみましょう．

In [4]:
extension = '.txt'
files = read_files(extension)

['data\\タンポポ花粉-１_1200倍(x1.2k).txt', 'data\\タンポポ花粉-１_200倍(x200).txt', 'data\\タンポポ花粉-１_600倍(x600).txt']


上記のようにリスト配列の結果がソートされて戻ります． 

# 3. 撮影メタデータを辞書化する

データ構造化では，ファイルに含まれいているヘッダー部と数値部を切り離す処理をします．その切り離す場所を判定するために特定の文字（単語）を指定する方法をとります．そのようなときに**特定の単語が現れる「行数」を判定**する関数です．

行判定は，次のような流れです．
1. ``open``関数で読みだしたファイルをtextに格納．
1. forループで一行づつtextを読み出しながら指定した文字（**word**）が含まれているかを``inキーワード``で判定
1. 一致した行をmatch_lineに格納
1. match_lineを戻し値としてreturnする

In [6]:
def get_keypara(_fn):

    """
    概要: KEYPARAFILEのcsvを読み込みメタデータを翻訳する
    @param _fn: templateファイル
    @return：　変換した語彙の辞書を辞書型として
    """
    
    _dict = dict()

    try:
        with open(_fn, mode='r') as f:
            line = f.readline()
            while line:
                str = line.strip().split(',')
                key = str[0].strip()
                value = str[1].strip()
                _dict[key] = value
                line = f.readline()
    except:
        print('get_keypara NG', file=sys.stderr)

    return _dict


予めcsvでまとめたキーパラメータのリスト``keypara.csv``を定義して次のように走らせます．

In [8]:
KEYPARAFILE = 'keypara.csv'
keypara = get_keypara(KEYPARAFILE)
keypara

{'InstructName': 'Instrument Model',
 'SerialNumber': 'Instrument S/N',
 'DataNumber': 'Comment and Number',
 'ImageName': 'File Name',
 'Date': 'Measurement Date',
 'Time': 'Measurement Time',
 'DataSize': 'Data Size',
 'PixelSize': 'Pixel Size',
 'SignalName': 'Signal Name',
 'AcceleratingVoltage': 'Accelerating Voltage',
 'Magnification': 'Magnification',
 'WorkingDistance': 'Working Distance'}

上記のようにリスト配列の結果が戻ります．  
１つ目は「data」フォルダーにある.rasのファイル名（相対パス形式），２つ目はファイル数，３つ目は拡張子（.ras）を省いたファイル名です．  
３番目はグラフのタイトルや出力ファイル名で使うために準備します．

# 4. ファイルの記述形式をcsv型に変える
データ構造化では，ファイルに含まれいているヘッダー部と数値部を切り離す処理をします．その切り離す場所を判定するために特定の文字（単語）を指定する方法をとります．そのようなときに**特定の単語が現れる「行数」を判定**する関数です．

行判定は，次のような流れです．
1. ``open``関数で読みだしたファイルをtextに格納．
1. forループで一行づつtextを読み出しながら指定した文字（**word**）が含まれているかを``inキーワード``で判定
1. 一致した行をmatch_lineに格納
1. match_lineを戻し値としてreturnする

In [9]:
def ini_to_csv(_fn):
    """
    概要: ini形式のtextファイルをcsv型へ変換する
    @param _fn: rawデータのファイル
    @return：　csv型をtxtとして戻す
    """
    
    csv = ""
    
    # encodingの判定
    with open(_fn, 'rb') as f:
        b = f.read()
        enc = detect(b)
        encd = enc['encoding']

    with open(_fn, mode='r', encoding=encd) as f:
        line = f.readline()
        
        while line:
            if not re.match(u"\[", line):
                str = line.rstrip(os.linesep).replace('=', ',', 1)
                csv += str
                csv += "\n"
            line = f.readline()
            
    return csv

ここでは「2. ファイルの拡張子を調べます」で出力したdataフォルタにある「XRD_RIGAKU.ras」について”＊RAS_INT_START”という文字（列）が含まれる行数を調べてみることにしましょう．  

単語判定ではword = '＊RAS_INT_START'として変数としておくと，関数には読み込みやすくなります．

In [10]:
filename = "data\\タンポポ花粉-１_1200倍(x1.2k).txt"
txt = ini_to_csv(filename)
txt

'InstructName,TM4000\nSemVersion,01-08\nSDMVersion,\nSerialNumber,2170A6-05\nDataNumber,TM4000 0001\nSampleName,\nFormat,tif\nImageName,タンポポ花粉-１_1200倍(x1.2k).tif\nDirectory,D:\\SemImage\\lixianglan\\\nDate,2021/06/23\nTime,16:43:33\nMedia,HD[Data]\nDataSize,1280x960\nPixelSize,82.68229\nSignalName,BSE\nDisplaySignalName,BSE\nSEDetSetting,\nAcceleratingVoltage,15000 Volt\nDecelerationVoltage,0 Volt\nDecelerationMode,\nMagnification,1200\nWorkingDistance,7920.5 um\nEmissionCurrent,46900 nA\nPhotoSize,1000\nMagnificationDisplay,0\nVacuum,50\nMicronMarker,40000\nSubMagnification,0\nSubSignalName,\nSpecimenBias,0 V\nCondencer1,9230\nScanSpeed,Capture_Slow(80)\nCaptureSpeed_Integration,\nCalibrationScanSpeed,24\nImgEnhance,0\nColorMode,Grayscale\nColorPalette,\nScreenMode,Full Screen\nComment,\nKeyWord1,\nKeyWord2,\nCondition,Vacc=15.0kV Mag=x1.20k WD=7.9mm LensMode=3\nDataDisplayCombine,1\nStageType,0\nStagePositionX,17500000\nStagePositionY,17500000\nStagePositionR,0.000\nStagePositionZ,0\

上記のように”469”という数字が現れました．これは'RAS_INT_START'という文字列（単語）が469行目にあることを示しています．（注意：pythonは0行からカウントが始まるため，1行目カウントであれば470行目となります）  

実は，この”RAS_INT_START”はリガクの.rasファイルをみると，数値部が始まる直前の単語であることがわかります．そうです．470行目からの数値部が回折角に対する強度データとなりますので，そのための判定に使っています．

では，XRD_RIGAKU.rasを直接指定してみましょう．dataframe形式で数値部が表示されます．

## 5. 数値化csvファイルの保存
上記のdataとして出力したdataframeをcsv化します．ファイルの保存は``pandas``の``to_csv``関数を使います．ここでは，"output"というフォルダに"test_extract.csv"というファイル名で保存します．

その時，``index =false``とすることで行番号（インデックス番号）の出力を不要としています．

In [13]:
def make_metatable (files,keypara):
    
    # 撮影メタデータの抽出
    n = 1
    for fn in files:
        txt = ""
        txt = ini_to_csv(fn)
        
        tmp = pd.read_csv(io.StringIO(txt), header=None,
                          names=['key', 'value%s' % n])
        if n == 1:
            out = tmp
        else:
            out = pd.merge(out, tmp, on='key', how='outer')
        n += 1
        
    out = out.set_index('key')
    out = out.transpose()
    out = out[list(keypara.keys())]
    d = dict(zip(list(keypara.keys()), list(keypara.values())))
    out = out.rename(columns=d)

    # 単位削除
    for index, row in out.iterrows():
        for k in ['Accelerating Voltage', 'Working Distance']:
            v = row[k]
            if v == v:
                v = v.split(' ')
                if len(v) > 1:
                    row[k] = v[0]

    # リストの出力
    out.to_csv('output/' + "metalist.csv", index = None,
               quoting=csv.QUOTE_NONNUMERIC, encoding='utf_8_sig')
    return print ('make meta-list file')    

In [14]:
filename = "data\\タンポポ花粉-１_1200倍(x1.2k).txt"
keypara = keypara
make_metatable (files,keypara)

make meta-list file


ここでは，第一引数として"data"とし，第二引数を＝”XRD_RIGAKU”として走らせることにしましょう．
なお，上記の最終行に``savefig``が見て取れるかと思います．ここで.pngの可視図を”output”フォルダーに保存していますのであわせて確認してみてください．

## 7. 一本化する
上記の一連の処理を一本化することをします．下記のように一本化したコードを``main``関数として定義しておきます．処理の順番を並べるだけです．
また，上記ではファイルが一つだけを想定していました．下記の``main``では複数の.rasファイルがある場合にでも実行できるよう``for``ループにしています．

計測では複数ファイルを一括して処理する場合がほとんどですので，このような一連の処理は``for``ループ化しておくと作業効率化に効いてきます．

In [15]:
def main():

    output_folder = 'output'
    os.makedirs(output_folder,exist_ok = True)
    
    extension = '.txt'
    files = read_files(extension)
    
    KEYPARAFILE = 'keypara.csv'
    keypara = get_keypara(KEYPARAFILE)
    
    
    make_metatable(files,keypara)

その``main``関数を下記のようにして走らせてみましょう．一連の処理が一貫で行われます．

In [16]:
main()

['data\\タンポポ花粉-１_1200倍(x1.2k).txt', 'data\\タンポポ花粉-１_200倍(x200).txt', 'data\\タンポポ花粉-１_600倍(x600).txt']
make meta-list file


## おわりに
このXRDは最も基本形のシンプルなタイプのデータ構造化の事例となります．基本的なポイントとなるのは次の処理項目です．

* データ構造化したいファイルを指定して読み込む
* 必要な数値部を抽出し保存する
* 数値部から可視化図を作成して保存する

あとは，お好みに応じて``make_figure``関数の設定を触りながらオリジナルのデザインを進めてゆくことで調整します．ここは``matplotlib``の勉強のたたき台にもなるかと思います．